# Day 11 of 100 days of Data Science

a crowdsourced Data Science learning program by Mr. Sharan

---

## Pandas: Aggregation and Group by

__Group by:__ GroupBy allows us to group our data based on different features and get a more accurate idea about your data.

__Aggregation:__ the agg() function allows multiple statistics to be calculated per group in one calculation. 

In [1]:
import pandas as pd
import dateutil

df = pd.read_csv('phone_data.csv')
df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   index         830 non-null    int64         
 1   date          830 non-null    datetime64[ns]
 2   duration      830 non-null    float64       
 3   item          830 non-null    object        
 4   month         830 non-null    object        
 5   network       830 non-null    object        
 6   network_type  830 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 45.5+ KB


In [2]:
df.describe()

,index,duration
count,830.000000,830.000000
mean,414.500000,117.804036
std,239.744656,444.129560
min,0.000000,1.000000
25%,207.250000,1.000000
50%,414.500000,24.500000
75%,621.750000,55.000000
max,829.000000,10528.000000


### Groupby()

A groupby function splits the data into groups, but won't return anything unless explicitly told to. That's why we use .count(), .sum() etc. It goes by __Split, Apply__ and __Combine__.

In [3]:
print(df.head(), "\n\t\t\t\t---")
df.groupby(['month']).groups.keys()

   index                date  duration  item    month   network network_type
0      0 2014-10-15 06:58:00    34.429  data  2014-11      data         data
1      1 2014-10-15 06:58:00    13.000  call  2014-11  Vodafone       mobile
2      2 2014-10-15 14:46:00    23.000  call  2014-11    Meteor       mobile
3      3 2014-10-15 14:48:00     4.000  call  2014-11     Tesco       mobile
4      4 2014-10-15 17:27:00     4.000  call  2014-11     Tesco       mobile 
				---


dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])

__The groupby() function returns a GroupBy object, but essentially describes how the rows of the original data set has been split. the GroupBy object .groups variable is a dictionary whose keys are the computed unique groups and corresponding values being the axis labels belonging to each group.__

In [4]:
df.groupby(['month']).groups['2015-01']

Int64Index([381, 386, 389, 390, 391, 392, 393, 394, 395, 396,
            ...
            583, 584, 585, 587, 588, 589, 590, 591, 592, 593],
           dtype='int64', length=205)

__Checking the first entries of each group__

In [5]:
df.groupby('month').first()

,index,date,duration,item,network,network_type
month,,,,,,
2014-11,0,2014-10-15 06:58:00,34.429,data,data,data
2014-12,228,2014-11-13 06:58:00,34.429,data,data,data
2015-01,381,2014-12-13 06:58:00,34.429,data,data,data
2015-02,577,2015-01-13 06:58:00,34.429,data,data,data
2015-03,729,2015-02-12 20:15:00,69.000,call,landline,landline


__Sum of the duration of each month group__

In [6]:
df.groupby(['month'])['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

__Counting the number of date entries in each month group__

In [7]:
df.groupby('month')['date'].count()

month
2014-11    230
2014-12    157
2015-01    205
2015-02    137
2015-03    101
Name: date, dtype: int64

__Counting the number of date entries in each item and month__

In [8]:
df.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

__The sum of durations, for calls only, in each network group__

In [9]:
df[df['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

__The output from a groupby and aggregation operation varies between Pandas Series and Pandas Dataframes, which can be confusing for new users. As a rule of thumb, if you calculate more than one column of results, your result will be a Dataframe. For a single column of results, the agg function, by default, will produce a Series.__

In [10]:
# produces Pandas Series
print(df.groupby('month')['duration'].sum())
# Produces Pandas DataFrame
print("\n",df.groupby('month')[['duration']].sum())

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

           duration
month             
2014-11  26639.441
2014-12  14641.870
2015-01  18223.299
2015-02  15522.299
2015-03  22750.441


__The groupby output will have an index or multi-index on rows corresponding to your chosen grouping variables. To avoid setting this index, pass “as_index=False” to the groupby operation.__

In [11]:
df.groupby('month', as_index=False).agg({"duration": "sum"})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


__Loops with Groupby__

In [12]:
loop = df.groupby('item')
loop.groups
for name, group in loop:
    print("Group Name: ", name, "Rows: ", group.shape[0])

Group Name:  call Rows:  388
Group Name:  data Rows:  150
Group Name:  sms Rows:  292


### Aggregation

### agg()
Instructions for aggregation are provided in the form of a python dictionary or list. The dictionary keys are used to specify the columns upon which you’d like to perform operations, and the dictionary values to specify the function to run. It uses __Aggregate, filter, transform, apply__ strategy

In [13]:
df.groupby(['month', 'item']).agg(
    {
         'duration':sum,    # Sum duration per group
         'network_type': "count",  # get the count of networks
         'date': 'first'  # get the first date per group
    })

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

__Applying multiple functions to columns in groups__

In [14]:
df.groupby(['month', 'item']).agg({
        # Find the min, max, and sum of the duration column
        'duration': [min, max, sum],
        # find the number of network type entries
        'network_type': "count",
        # minimum, first, and number of unique dates
        'date': [min, 'first', 'nunique']
    })

duration                       network_type                date  \
                  min        max        sum        count                 min   
month   item                                                                   
2014-11 call    1.000   1940.000  25547.000          107 2014-10-15 06:58:00   
        data   34.429     34.429    998.441           29 2014-10-15 06:58:00   
        sms     1.000      1.000     94.000           94 2014-10-16 22:18:00   
2014-12 call    2.000   2120.000  13561.000           79 2014-11-14 17:24:00   
        data   34.429     34.429   1032.870           30 2014-11-13 06:58:00   
        sms     1.000      1.000     48.000           48 2014-11-14 17:28:00   
2015-01 call    2.000   1859.000  17070.000           88 2014-12-15 20:03:00   
        data   34.429     34.429   1067.299           31 2014-12-13 06:58:00   
        sms     1.000      1.000     86.000           86 2014-12-15 19:56:00   
2015-02 call    1.000   1863.000  14416.000           67 2015-01-15 10:36:00   
        data   34.429     34.429   1067.299           31 2015-01-13 06:58:00   
        sms     1.000      1.000     39.000           39 2015-01-15 12:23:00   
2015-03 call    2.000  10528.000  21727.000           47 2015-02-12 20:15:00   
        data   34.429     34.429    998.441           29 2015-02-13 06:58:00   
        sms     1.000      1.000     25.000           25 2015-02-19 18:46:00   

                                          
                           first nunique  
month   item                              
2014-11 call 2014-10-15 06:58:00     104  
        data 2014-10-15 06:58:00      29  
        sms  2014-10-16 22:18:00      79  
2014-12 call 2014-11-14 17:24:00      76  
        data 2014-11-13 06:58:00      30  
        sms  2014-11-14 17:28:00      41  
2015-01 call 2014-12-15 20:03:00      84  
        data 2014-12-13 06:58:00      31  
        sms  2014-12-15 19:56:00      58  
2015-02 call 2015-01-15 10:36:00      67  
        data 2015-01-13 06:58:00      31  
        sms  2015-01-15 12:23:00      27  
2015-03 call 2015-02-12 20:15:00      47  
        data 2015-02-13 06:58:00      29  
        sms  2015-02-19 18:46:00      17

__Renaming Group Aggregation columns using NamedAgg()__

In [15]:
df[df['item'] == 'call'].groupby('month').agg(
        # Find the min, max, of the duration column
        max_dur=pd.NamedAgg(column='duration', aggfunc= max),
        min_dur=pd.NamedAgg(column = 'duration', aggfunc = min)
    )

,max_dur,min_dur
month,,
2014-11,1940.0,1.0
2014-12,2120.0,2.0
2015-01,1859.0,2.0
2015-02,1863.0,1.0
2015-03,10528.0,2.0


__Tuple Named Aggregation__

In [16]:
df[df['item'] == 'call'].groupby('month').agg(
    # Get max of the duration column for each group
    max_duration=('duration', max),
    # Get min of the duration column for each group
    min_duration=('duration', min),
    # Get sum of the duration column for each group
    total_duration=('duration', sum),
)

,max_duration,min_duration,total_duration
month,,,
2014-11,1940.0,1.0,25547.0
2014-12,2120.0,2.0,13561.0
2015-01,1859.0,2.0,17070.0
2015-02,1863.0,1.0,14416.0
2015-03,10528.0,2.0,21727.0


__Renaming the columns using droplevel()__

In [17]:
grouped = df.groupby('month').agg({"duration":[min, max]})
grouped.columns = grouped.columns.droplevel(level=0)
print(grouped.rename(columns={"min":"min_dur", "max":"max_dur"}))

         min_dur  max_dur
month                    
2014-11      1.0   1940.0
2014-12      1.0   2120.0
2015-01      1.0   1859.0
2015-02      1.0   1863.0
2015-03      1.0  10528.0


__Renaming the indexes using ravel()__

In [18]:
grouped = df.groupby('month').agg({"duration": [min, max]}) 
# Using ravel, and a string join, we can create better names for the columns:
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
grouped.head()

,duration_min,duration_max
month,,
2014-11,1.0,1940.0
2014-12,1.0,2120.0
2015-01,1.0,1859.0
2015-02,1.0,1863.0
2015-03,1.0,10528.0


__Note:__ There are 3 ways to aggregate, by using __Lists *df[column].agg([func,func])* , Dictionary *df.agg({column:[func,func],..})* and Tuple *df.agg(name=(column,func),..)*__

## Thank you!